In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('dataset-2.csv')
df

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1
5,1001410,1001412,15.6
6,1001412,1001414,18.2
7,1001414,1001416,13.2
8,1001416,1001418,13.6
9,1001418,1001420,12.9


In [5]:
df.corr()

,id_start,id_end,distance
id_start,1.000000,0.649080,-0.442861
id_end,0.649080,1.000000,-0.390121
distance,-0.442861,-0.390121,1.000000


In [6]:
def calculate_distance_matrix(file_path):
    
    df = pd.read_csv(file_path)
    
    df_pivot = df.pivot(index='id_start', columns='id_end', values='distance')
    distance_matrix = df_pivot.add(df_pivot.transpose(), fill_value=0)
    np.fill_diagonal(distance_matrix.values, 0)
    for i in range( 0 ,len(distance_matrix)):
        for j in range( 0, len(distance_matrix)):
            if np.isnan(distance_matrix.iloc[i, j]):
                distance_matrix.iloc[i, j] = distance_matrix.iloc[i, :j].add(distance_matrix.iloc[:j, j]).min()
                distance_matrix.iloc[j, i] = distance_matrix.iloc[i, j]

    return distance_matrix

distance_matrix = calculate_distance_matrix('dataset-2.csv')
print(distance_matrix.head())


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   

         1001414  1001416  1001418  ...  1001462  1001464  1001466  1001468  \
1001400    112.5    125.7    139.3  ...      NaN      NaN      NaN      NaN   
1001402    102.8    116.0    129.6  ...      NaN      NaN      NaN      NaN   
1001404     82.6     95.8    109.4  ...      NaN      NaN      NaN      NaN   
1001406     66.6     79.8     93.4  ...      NaN      NaN      NaN      NaN   
1001408     44.9     58.1     71.7  ...      NaN      NaN      NaN      NaN   

         1001470  1001472  1001488  1004354  1004355  1004356  
1001400      NaN

In [7]:
da=distance_matrix.iloc[0:9,0:9]
da

,1001400,1001402,1001404,1001406,1001408,1001410,1001412,1001414,1001416
1001400,0.0,9.7,29.9,45.9,67.6,78.7,94.3,112.5,125.7
1001402,9.7,0.0,20.2,36.2,57.9,69.0,84.6,102.8,116.0
1001404,29.9,20.2,0.0,16.0,37.7,48.8,64.4,82.6,95.8
1001406,45.9,36.2,16.0,0.0,21.7,32.8,48.4,66.6,79.8
1001408,67.6,57.9,37.7,21.7,0.0,11.1,26.7,44.9,58.1
1001410,78.7,69.0,48.8,32.8,11.1,0.0,15.6,33.8,47.0
1001412,94.3,84.6,64.4,48.4,26.7,15.6,0.0,18.2,31.4
1001414,112.5,102.8,82.6,66.6,44.9,33.8,18.2,0.0,13.2
1001416,125.7,116.0,95.8,79.8,58.1,47.0,31.4,13.2,0.0


In [10]:
def unroll_distance_matrix(distance_matrix):
   
    long_df = distance_matrix.reset_index().melt(id_vars='index', var_name='id_start', value_name='distance')
   
    long_df.rename(columns={'index': 'id_end'}, inplace=True)
    
   
    long_df = long_df[long_df['id_start'] != long_df['id_end']]
    
    
    long_df = long_df[long_df['distance'] > 0]

    
    long_df = long_df[['id_start','id_end',  'distance']]

    return long_df

unrolled_df = unroll_distance_matrix(da)
print(unrolled_df)


   id_start   id_end  distance
1   1001400  1001402       9.7
2   1001400  1001404      29.9
3   1001400  1001406      45.9
4   1001400  1001408      67.6
5   1001400  1001410      78.7
..      ...      ...       ...
75  1001416  1001406      79.8
76  1001416  1001408      58.1
77  1001416  1001410      47.0
78  1001416  1001412      31.4
79  1001416  1001414      13.2

[72 rows x 3 columns]


In [11]:
unrolled_df.head(9)

,id_start,id_end,distance
1,1001400,1001402,9.7
2,1001400,1001404,29.9
3,1001400,1001406,45.9
4,1001400,1001408,67.6
5,1001400,1001410,78.7
6,1001400,1001412,94.3
7,1001400,1001414,112.5
8,1001400,1001416,125.7
9,1001402,1001400,9.7


In [14]:
def find_ids_within_ten_percentage_threshold(df, reference_id):
    
    reference_avg_distance = df[df['id_start'] == reference_id]['distance'].mean()

    lower_threshold = reference_avg_distance * 0.9
    upper_threshold = reference_avg_distance * 1.1

    ids_within_threshold = df.groupby('id_start').filter(lambda x: lower_threshold <= x['distance'].mean() <= upper_threshold)

    return sorted(ids_within_threshold['id_start'].unique())
reference_id = 1001416 
result = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print(result)

[1001400, 1001416]


In [16]:
def calculate_toll_rate(df):
    
    rates = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    for vehicle, rate in rates.items():
        df[vehicle] = df['distance'] * rate
    
    return df

toll_rate_df = calculate_toll_rate(unrolled_df)
print(toll_rate_df)


   id_start   id_end  distance   moto    car      rv     bus   truck
1   1001400  1001402       9.7   7.76  11.64   14.55   21.34   34.92
2   1001400  1001404      29.9  23.92  35.88   44.85   65.78  107.64
3   1001400  1001406      45.9  36.72  55.08   68.85  100.98  165.24
4   1001400  1001408      67.6  54.08  81.12  101.40  148.72  243.36
5   1001400  1001410      78.7  62.96  94.44  118.05  173.14  283.32
..      ...      ...       ...    ...    ...     ...     ...     ...
75  1001416  1001406      79.8  63.84  95.76  119.70  175.56  287.28
76  1001416  1001408      58.1  46.48  69.72   87.15  127.82  209.16
77  1001416  1001410      47.0  37.60  56.40   70.50  103.40  169.20
78  1001416  1001412      31.4  25.12  37.68   47.10   69.08  113.04
79  1001416  1001414      13.2  10.56  15.84   19.80   29.04   47.52

[72 rows x 8 columns]


In [17]:
from datetime import time, timedelta

def calculate_time_based_toll_rates(df):
    
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    discounts = {
        'weekdays': {
            'morning': (time(0, 0), time(10, 0), 0.8),
            'day': (time(10, 0), time(18, 0), 1.2),
            'evening': (time(18, 0), time(23, 59), 0.8),
        },
        'weekends': (time(0, 0), time(23, 59), 0.7)
    }

    rows = []

    for _, row in df.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']
        distances = row[['moto', 'car', 'rv', 'bus', 'truck']].values

        for day in days_of_week:
            for interval in discounts['weekdays'].values():
                start_time, end_time, factor = interval
                adjusted_rates = distances * factor
                rows.append([day, start_time, day, end_time] + list(adjusted_rates))

            for day in ['Saturday', 'Sunday']:
                start_time, end_time, factor = discounts['weekends']
                adjusted_rates = distances * factor
                rows.append([day, start_time, day, end_time] + list(adjusted_rates))

    columns = ['start_day', 'start_time', 'end_day', 'end_time', 'moto', 'car', 'rv', 'bus', 'truck']
    result_df = pd.DataFrame(rows, columns=columns)

    return result_df


time_based_toll_df = calculate_time_based_toll_rates(toll_rate_df)
print(time_based_toll_df)


     start_day start_time   end_day  end_time    moto     car      rv     bus  \
0       Monday   00:00:00    Monday  10:00:00   6.208   9.312  11.640  17.072   
1       Monday   10:00:00    Monday  18:00:00   9.312  13.968  17.460  25.608   
2       Monday   18:00:00    Monday  23:59:00   6.208   9.312  11.640  17.072   
3     Saturday   00:00:00  Saturday  23:59:00   5.432   8.148  10.185  14.938   
4       Sunday   00:00:00    Sunday  23:59:00   5.432   8.148  10.185  14.938   
...        ...        ...       ...       ...     ...     ...     ...     ...   
2515    Sunday   00:00:00    Sunday  10:00:00   8.448  12.672  15.840  23.232   
2516    Sunday   10:00:00    Sunday  18:00:00  12.672  19.008  23.760  34.848   
2517    Sunday   18:00:00    Sunday  23:59:00   8.448  12.672  15.840  23.232   
2518  Saturday   00:00:00  Saturday  23:59:00   7.392  11.088  13.860  20.328   
2519    Sunday   00:00:00    Sunday  23:59:00   7.392  11.088  13.860  20.328   

       truck  
0     27.936